In [1]:
import os
from enum import Enum
import re
import numpy as np
from pyarabic.araby import separate, tokenize, is_arabicrange, strip_tashkeel, strip_tatweel
import nltk
from nltk.tokenize import sent_tokenize
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential,load_model
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from gensim.models import Word2Vec

from chars_enums import *
from file_reader import FileReader
from preprocessor import Preprocessor

2023-12-31 11:50:59.636201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-31 11:50:59.636264: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-31 11:50:59.637224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-31 11:50:59.643515: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 11:51:00.678279: W tensorflow/compiler/tf2

In [2]:
P1 = Preprocessor()
arabic_text = "الذِّمِّيِّ أَنْ يَحْتَسِبَ عَلَى الْمُسْلِم"
result, diacritics = P1.separate_diacritics(arabic_text)
print(result)
print(diacritics)

for diacritic_value in diacritics:
    if diacritic_value in [diacritic.value.decode("utf-8") for diacritic in ArabicDiacritics]:
        for diacritic in ArabicDiacritics:
            if diacritic.value.decode("utf-8") == diacritic_value:
                print(f"Extracted Diacritic: {diacritic.name}")
    else:
        print(f"Extracted Diacritic: No Diacritic")

الذمي أن يحتسب على المسلم
['', '', 'ِّ', 'ِّ', 'ِّ', '', 'َ', 'ْ', '', 'َ', 'ْ', 'َ', 'ِ', 'َ', '', 'َ', 'َ', '', '', '', 'ْ', 'ُ', 'ْ', 'ِ', '']
Extracted Diacritic: No Diacritic
Extracted Diacritic: No Diacritic
Extracted Diacritic: ARABIC_SHADDA_KASRA
Extracted Diacritic: ARABIC_SHADDA_KASRA
Extracted Diacritic: ARABIC_SHADDA_KASRA
Extracted Diacritic: No Diacritic
Extracted Diacritic: ARABIC_FATHA
Extracted Diacritic: ARABIC_SUKUN
Extracted Diacritic: No Diacritic
Extracted Diacritic: ARABIC_FATHA
Extracted Diacritic: ARABIC_SUKUN
Extracted Diacritic: ARABIC_FATHA
Extracted Diacritic: ARABIC_KASRA
Extracted Diacritic: ARABIC_FATHA
Extracted Diacritic: No Diacritic
Extracted Diacritic: ARABIC_FATHA
Extracted Diacritic: ARABIC_FATHA
Extracted Diacritic: No Diacritic
Extracted Diacritic: No Diacritic
Extracted Diacritic: No Diacritic
Extracted Diacritic: ARABIC_SUKUN
Extracted Diacritic: ARABIC_DAMMA
Extracted Diacritic: ARABIC_SUKUN
Extracted Diacritic: ARABIC_KASRA
Extracted Diacrit

### Model Training